In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [44]:
df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [45]:
df = df.dropna()


In [46]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in df.select_dtypes(include="object"):
    df[col] = encoder.fit_transform(df[col])

In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])

In [48]:
X = df.drop("Sleep Disorder", axis=1).values
y = df["Sleep Disorder"].values

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [50]:
import numpy as np
y_test_binary = np.where(y_test == "Yes", 1, 0)

In [51]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [52]:
def logistic_loss(y, y_pred):
    y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

In [53]:
def logistic_regression_train(X, y, lr=0.01, epochs=2000):
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)

    for _ in range(epochs):
        linear_output = X @ theta
        y_pred = sigmoid(linear_output)

        gradient = (1/n_samples) * (X.T @ (y_pred - y))
        theta -= lr * gradient

    return theta

In [54]:
def logistic_regression_predict(X, theta):
    probs = sigmoid(X @ theta)
    return (probs > 0.5).astype(int)

In [55]:
theta = logistic_regression_train(X_train, y_train, lr=0.01, epochs=3000)

In [56]:
y_pred_binary = (y_pred >= 0.5).astype(int)


In [57]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_test_binary, y_pred_binary))
print("Precision:", precision_score(y_test_binary, y_pred_binary))
print("Recall:", recall_score(y_test_binary, y_pred_binary))
print("F1 Score:", f1_score(y_test_binary, y_pred_binary))

Accuracy: 0.5161290322580645
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [58]:
def logistic_regression_L2(X, y, lr=0.01, epochs=2000, lam=0.1):
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)

    for _ in range(epochs):
        y_pred = sigmoid(X @ theta)

        gradient = (1/n_samples) * (X.T @ (y_pred - y)) + lam * theta
        theta -= lr * gradient

    return theta

In [59]:
def logistic_regression_L1(X, y, lr=0.01, epochs=2000, lam=0.1):
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)

    for _ in range(epochs):
        y_pred = sigmoid(X @ theta)

        gradient = (1/n_samples) * (X.T @ (y_pred - y)) + lam * np.sign(theta)
        theta -= lr * gradient

    return theta

In [60]:
def logistic_regression_elasticnet(X, y, lr=0.01, epochs=2000, l1=0.1, l2=0.1):
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)

    for _ in range(epochs):
        y_pred = sigmoid(X @ theta)

        gradient = (1/n_samples) * (X.T @ (y_pred - y)) + l1*np.sign(theta) + l2*theta
        theta -= lr * gradient

    return theta